#### Part 1: Create a framework for the game mechanics/rules ####

In [ ]:
import numpy as np
import time
import os

def othello_game(board_size: int = 8):

    if not (isinstance(board_size, int) and board_size > 0 and board_size % 2 == 0):
        raise ValueError("board_size must be an even positive integer.")

    board = np.full((board_size, board_size), '.', dtype=str)
    mid = board_size // 2
    board[mid-1][mid-1] = board[mid][mid] = 'B' 
    board[mid-1][mid] = board[mid][mid-1] = 'W'

    def clear_screen():
        try:
            from IPython.display import clear_output
            clear_output(wait=True)
        except Exception:
            # Fallback for regular terminals
            if os.name == 'nt':
                _ = os.system('cls')
            else:
                _ = os.system('clear')

    def print_board(board):
        n = len(board)

        GREEN = "\033[32m"
        BLUE = "\033[34m"
        BLACK = "\033[30m"
        WHITE = "\033[97m"
        RESET = "\033[0m"
        GREEN_BG = "\033[42m"   

        TL, TM, TR = GREEN + '┌' + RESET, GREEN + '┬' + RESET, GREEN + '┐' + RESET
        ML, MM, MR = GREEN + '├' + RESET, GREEN + '┼' + RESET, GREEN + '┤' + RESET
        BL, BM, BR = GREEN + '└' + RESET, GREEN + '┴' + RESET, GREEN + '┘' + RESET
        HL = GREEN + '─' + RESET
        VT = GREEN + '│' + RESET

        label_w = max(2, len(str(n)))
        cell_inner = 5      
        cell_h = 1       

        hor = HL * cell_inner

        top_border = TL + (hor + TM) * (n - 1) + hor + TR
        mid_border = ML + (hor + MM) * (n - 1) + hor + MR
        bot_border = BL + (hor + BM) * (n - 1) + hor + BR

        cols = [chr(ord('A') + i) for i in range(n)]
        header = ' ' * (label_w + 2)
        header += ' '.join(BLUE + col.center(cell_inner) + RESET for col in cols)
        print()
        print(header)

        print(' ' * (label_w + 1) + top_border)

        mid_line_index = cell_h // 2
        for r in range(n):
            for inner_row in range(cell_h):
                if inner_row == mid_line_index:
                    row_label = BLUE + str(r + 1).rjust(label_w) + ' ' + RESET
                else:
                    row_label = ' ' * (label_w + 1)

                row_cells = []
                for c in range(n):
                    val = board[r][c]
                    if inner_row == mid_line_index:
                        if val == 'B':
                            ch = GREEN_BG + BLACK + ' ● ' + RESET
                        elif val == 'W':
                            ch = GREEN_BG + WHITE + ' ● ' + RESET
                        else:
                            ch = GREEN_BG + '   ' + RESET
                        cell_text = ' ' + ch.center(cell_inner) + ' '
                    else:
                        cell_text = GREEN_BG + '   ' * (cell_inner +2) + RESET
                    row_cells.append(cell_text)

                print(row_label + VT + VT.join(row_cells) + VT)

            if r < n - 1:
                print(' ' * (label_w + 1) + mid_border)
            else:
                print(' ' * (label_w + 1) + bot_border)

    def is_valid_move(x, y, player):
        if board[x][y] != '.':
            return False
        opponent = 'W' if player == 'B' else 'B'
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            found_opponent = False
            while 0 <= nx < board_size and 0 <= ny < board_size:
                if board[nx][ny] == opponent:
                    found_opponent = True
                elif board[nx][ny] == player:
                    if found_opponent:
                        return True
                    else:
                        break
                else:
                    break
                nx += dx
                ny += dy
        return False

    def make_move(x, y, player):
        board[x][y] = player
        opponent = 'W' if player == 'B' else 'B'
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            to_flip = []
            while 0 <= nx < board_size and 0 <= ny < board_size:
                if board[nx][ny] == opponent:
                    to_flip.append((nx, ny))
                elif board[nx][ny] == player:
                    for fx, fy in to_flip:
                        board[fx][fy] = player
                    break
                else:
                    break
                nx += dx
                ny += dy

    current_player = 'B'
    
    while True:
        clear_screen()
        b_count = np.sum(board == 'B')
        w_count = np.sum(board == 'W')
        black_moves = any(is_valid_move(r,c,'B') for r in range(board_size) for c in range(board_size))
        white_moves = any(is_valid_move(r,c,'W') for r in range(board_size) for c in range(board_size))
        if not black_moves and not white_moves:
            print_board(board)
            print("\nGame over!")
            if b_count > w_count:
                print(f"\nBlack wins {b_count}-{w_count}!")
            elif w_count > b_count:
                print(f"\nWhite wins {w_count}-{b_count}!")
            else:
                print(f"\nIt's a tie {b_count}-{w_count}!")
            break
        print_board(board)
        print(f"\nCurrent player: {current_player}")
        print(f"\nCurrent counts: B = {b_count},  W = {w_count}.")
        legal_moves = [(r,c) for r in range(board_size) for c in range(board_size) if is_valid_move(r,c,current_player)]
        if not legal_moves:
            print(f"\n{current_player} has no legal moves — turn passed.")
            time.sleep(2)
            current_player = 'W' if current_player == 'B' else 'B'
            continue
        move = input("Enter your move coordinate or 'end' to quit: ")
        if move.lower() == 'end':
            print("\nGame ended.")
            if b_count > w_count:
                print(f"\nBlack wins {b_count}-{w_count}!")
            elif w_count > b_count:
                print(f"\nWhite wins {w_count}-{b_count}!")
            else:
                print(f"\nIt's a tie {b_count}-{w_count}!")
            time.sleep(2)
            break
        try:
            if not move:
                raise ValueError("empty move")
            letter = move[0]
            number = move[1:]
            letter = letter.upper()
            if letter < 'A' or letter > chr(ord('A') + board_size - 1):
                raise ValueError("\nRow letter out of range")
            y = ord(letter) - ord('A')  
            x = int(number) - 1
            if is_valid_move(x, y, current_player):
                make_move(x, y, current_player)
                current_player = 'W' if current_player == 'B' else 'B'
            else:
                print("\nInvalid move. Try again.")
                time.sleep(2)
        except (ValueError, IndexError):
            print("\nInvalid input. Please enter valid row and column values.")
            time.sleep(2)


In [ ]:
othello_game(4)

#### Part 2a: Reinforcement Learning Experiment on a 4x4 Grid ####

Recreated the game without any visual elements, and no capacity for dealing with errors. This game will just take correct inputs until the game is over and will output 1 for a White win, 0 for a White loss, 0.5 for a tie, and Error for any errors.

In [146]:
import numpy as np

def othello_reinforced(board_size: int = 8):

    if not (isinstance(board_size, int) and board_size > 0 and board_size % 2 == 0):
        raise ValueError("board_size must be an even positive integer.")

    board = np.full((board_size, board_size), '.', dtype=str)
    mid = board_size // 2
    board[mid-1][mid-1] = board[mid][mid] = 'B' 
    board[mid-1][mid] = board[mid][mid-1] = 'W'

    def is_valid_move(x, y, player):
        if board[x][y] != '.':
            return False
        opponent = 'W' if player == 'B' else 'B'
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            found_opponent = False
            while 0 <= nx < board_size and 0 <= ny < board_size:
                if board[nx][ny] == opponent:
                    found_opponent = True
                elif board[nx][ny] == player:
                    if found_opponent:
                        return True
                    else:
                        break
                else:
                    break
                nx += dx
                ny += dy
        return False

    def make_move(x, y, player):
        board[x][y] = player
        opponent = 'W' if player == 'B' else 'B'
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            to_flip = []
            while 0 <= nx < board_size and 0 <= ny < board_size:
                if board[nx][ny] == opponent:
                    to_flip.append((nx, ny))
                elif board[nx][ny] == player:
                    for fx, fy in to_flip:
                        board[fx][fy] = player
                    break
                else:
                    break
                nx += dx
                ny += dy

    current_player = 'B'
    
    while True:
        b_count = np.sum(board == 'B')
        w_count = np.sum(board == 'W')
        black_moves = any(is_valid_move(r,c,'B') for r in range(board_size) for c in range(board_size))
        white_moves = any(is_valid_move(r,c,'W') for r in range(board_size) for c in range(board_size))
        if not black_moves and not white_moves:
            break
        legal_moves = [(r,c) for r in range(board_size) for c in range(board_size) if is_valid_move(r,c,current_player)]
        if not legal_moves:
            current_player = 'W' if current_player == 'B' else 'B'
            continue
        move = input()
        try:
            if not move:
                return "Error"
            letter = move[0]
            number = move[1:]
            letter = letter.upper()
            if letter < 'A' or letter > chr(ord('A') + board_size - 1):
                return "Error"
            y = ord(letter) - ord('A')  
            x = int(number) - 1
            if is_valid_move(x, y, current_player):
                make_move(x, y, current_player)
                current_player = 'W' if current_player == 'B' else 'B'
            else:
                return "Error"
        except (ValueError, IndexError):
            return "Error"
    winner = 1 if w_count > b_count else 0 if b_count > w_count else 0.5
    return winner


Experimenting with running through the basic code for the game with random iterations of moves on a very small grid - a brute force approach to finding all the possible wins for White in a 4x4 grid. To lower the number of permutations even more, I will start with the moves C1 then D1, to lower the number of possible moves to O(10^6)

In [ ]:
from itertools import permutations
from unittest.mock import patch

possible_moves = ['A1', 'A2', 'A3', 'A4', 'B1', 'B4', 'C4', 'D2', 'D3', 'D4']

def brute_force_othello(moves):

    error_count = 0
    win_count = 0
    loss_count = 0
    tie_count = 0
    misc_count = 0

    for perm in permutations(moves):
        move_list = ['C1', 'D1'] + list(perm)
        with patch('builtins.input', side_effect=move_list):
            result = othello_reinforced(4)
            if result == "Error":
                error_count += 1
            elif result == 1:
                win_count += 1
            elif result == 0:
                loss_count += 1
            elif result == 0.5:
                tie_count += 1
            else:
                misc_count += 1

    return win_count, loss_count, tie_count, error_count, misc_count

brute_force_othello(possible_moves)


(3172, 446, 174, 3625008)

Therefore ['C1', 'D1'] start:

(win, loss, tie, error) = (3172, 446, 174, 3625008)

i.e. With the starting moves 1. B C1, 2. W D1, the possible results for White are 3172-446-174 (W-L-D)

Making some changes for efficiency purposes:
- Introduce lists as inputs for the Othello function to avoid difficulties with automatic inputs
- Remove lists that contain erroneous beginnings that have already been identified as erroneous to avoid repeats
- Include more debugging techniques to allow for more transparent code running

In [162]:
import numpy as np

def othello_streamlined_4x4(moves: list = []):

    board_size = 4
    board = np.full((board_size, board_size), '.', dtype=str)
    mid = board_size // 2
    board[mid-1][mid-1] = board[mid][mid] = 'B' 
    board[mid-1][mid] = board[mid][mid-1] = 'W'

    def is_valid_move(x, y, player):
        if board[x][y] != '.':
            return False
        opponent = 'W' if player == 'B' else 'B'
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            found_opponent = False
            while 0 <= nx < board_size and 0 <= ny < board_size:
                if board[nx][ny] == opponent:
                    found_opponent = True
                elif board[nx][ny] == player:
                    if found_opponent:
                        return True
                    else:
                        break
                else:
                    break
                nx += dx
                ny += dy
        return False

    def make_move(x, y, player):
        board[x][y] = player
        opponent = 'W' if player == 'B' else 'B'
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            to_flip = []
            while 0 <= nx < board_size and 0 <= ny < board_size:
                if board[nx][ny] == opponent:
                    to_flip.append((nx, ny))
                elif board[nx][ny] == player:
                    for fx, fy in to_flip:
                        board[fx][fy] = player
                    break
                else:
                    break
                nx += dx
                ny += dy

    current_player = 'B'
    i = 0
    
    while True:
        b_count = np.sum(board == 'B')
        w_count = np.sum(board == 'W')
        black_moves = any(is_valid_move(r,c,'B') for r in range(board_size) for c in range(board_size))
        white_moves = any(is_valid_move(r,c,'W') for r in range(board_size) for c in range(board_size))
        if not black_moves and not white_moves:
            break
        legal_moves = [(r,c) for r in range(board_size) for c in range(board_size) if is_valid_move(r,c,current_player)]
        if not legal_moves:
            current_player = 'W' if current_player == 'B' else 'B'
            continue
        move = moves[i]
        try:
            letter = move[0]
            number = move[1:]
            letter = letter.upper()
            y = ord(letter) - ord('A')  
            x = int(number) - 1
            if is_valid_move(x, y, current_player):
                make_move(x, y, current_player)
                current_player = 'W' if current_player == 'B' else 'B'
            else:
                return 'E' + str(i)
        except (ValueError, IndexError):
            return 'E' + str(i)
        i += 1
    winner = 1 if w_count > b_count else 0 if b_count > w_count else 0.5
    return winner


In [168]:
possible_moves = ['C1', 'D1', 'D2', 'chocolate', 'D4', 'A1', 'B1', 'B4', 'C4', 'D2', 'D3', 'D4']


result = othello_streamlined_4x4(possible_moves)

if isinstance(result, str):
    error_end = result[1:]
    print(f"Error at move index: {error_end}")

Error at move index: 3


So same again with C1 D1 as first 2 moves, but this time with many improvements:

In [190]:
from itertools import permutations

def brute_othello_4x4():

    moves = ['A1', 'A2', 'A3', 'A4', 'B1', 'B4', 'C4', 'D2', 'D3', 'D4']

    win_count = 0
    loss_count = 0
    tie_count = 0
    misc_count = 0
    forbidden_prefixes = []

    def is_prefix(pref, perm):
        len_p = len(pref)
        return tuple(perm[:len_p]) == tuple(pref)

    for perm in permutations(moves):

        current_perm = list(perm)
        move_list = ['C1', 'D1'] + current_perm

        skip = False

        for pref in forbidden_prefixes:
            if is_prefix(pref, current_perm):
                skip = True
                break

        if not skip:
            result = othello_streamlined_4x4(move_list)

            if isinstance(result, str):
                error_end = int(result[1:]) - 2
                error_prefix = current_perm[0:error_end+1]
                forbidden_prefixes.append(error_prefix)

            elif result == 1:
                win_count += 1
            elif result == 0:
                loss_count += 1
            elif result == 0.5:
                tie_count += 1
            else:
                misc_count += 1

    return win_count, loss_count, tie_count, misc_count


In [191]:
brute_othello_4x4()

(3172, 446, 174, 0)

Saved 5 mins but this method required going through every permutation and then checking if the error had shown up, which really isn't any quicker than just running the game because the game is actually pretty quick. Need some way of actually removing the permutations that we don't want to see. Don't think this permutations package is going to work for what I want to do.